In [ ]:
import os
import os.path as op
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import seaborn as sns
sns.set_theme(style="darkgrid", color_codes=True)
sns.set(font_scale=1.35, style="ticks") #set styling preferences
import statsmodels.api as sm
from scipy import stats
import math
from math import pi
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from scipy.spatial.distance import cdist
from scipy.cluster.vq import kmeans2,vq, whiten
import geopandas as gpd
import h5py
import boto.s3
import glob
import boto3
from zipfile import ZipFile
import shutil

In [ ]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
# Showing the entire number in dataframe
pd.set_option('float_format', '{:f}'.format)

In [ ]:
%%time
s3 = boto3.client("s3")
key = "pilates-outputs/sfbay-baseline-2022124/inexus/sfbay_baseline_default-1.0_2020__20221224.csv.gz"  #the path should be updated
obj = s3.get_object(Bucket="beam-outputs", Key=key)
sfbase = pd.read_csv(obj['Body'], compression = 'gzip',index_col='Unnamed: 0')

In [ ]:
sfbase = sfbase[(sfbase['duration_travelling'] > 0)].reset_index(drop=True)

### We used the Isolation Forest method to remove the outliers 

In [ ]:
from sklearn.ensemble import IsolationForest
# Select the columns you want to use for outlier detection
X = sfbase[['Potential_INEXUS_in_dollar_2023', 'Realized_INEXUS_in_dollar_2023', 'duration_walking']].values

# Define the Isolation Forest model
isof = IsolationForest(n_estimators=100, contamination=0.1, random_state=42)

# Fit the model to the data
isof.fit(X)

# Use the model to predict the outliers
y_pred = isof.predict(X)

# Add the outlier predictions to the DataFrame
sfbase['is_outlier'] = y_pred

# Remove the outliers from the DataFrame
sfbase_clean_ml_walk = sfbase[sfbase['is_outlier'] == 1]

In [ ]:
plt.boxplot(sfbase_clean_ml_walk['Realized_INEXUS_in_dollar_2023'], whis=3)
plt.show()

In [ ]:
plt.boxplot(sfbase_clean_ml_walk['duration_walking'], whis=3)
plt.show()

### Baseline INEXUS plots (ridehail price scenario)

#### Simple Potential INEXUS

In [ ]:
from matplotlib.ticker import FuncFormatter
sns.set(rc={'figure.figsize':(20,14)})
sns.set_context('talk')

sns.set_style("whitegrid",{'grid.color': 'gainsboro'}) 
#'whitesmoke': a very light grey color with a hint of blue.
#'gainsboro': a very light grey color with a hint of blue-green.
#'lightgray': a slightly lighter shade of grey than 'lightgrey'.
#'lavender'

# Define function to format y-axis labels as percentages
def to_percent(y, position):
    return "{:.1f}%".format(y * 100)

ax=sns.kdeplot(data=sfbase_clean_ml_walk, x="Potential_INEXUS_in_dollar_2023", 
            fill=True, alpha=0.1, color="#CC3311", bw_adjust =15, linewidth =5)

# Format y-axis labels as percentages
formatter = FuncFormatter(to_percent)
ax.yaxis.set_major_formatter(formatter)

#sns.move_legend(ax, labels = ['Lowest 10% Income','Highest 10% Income'], loc="right", bbox_to_anchor=(1.25, 0.6), ncol=1, title=None, frameon=True)
plt.legend(labels = ['Baseline Scenario'],
          fontsize='small', fancybox=False, ncol=1, frameon=True, loc = 'best', prop={'size': 32}) #title="Modes" #bbox_to_anchor=(1.5, 0.7), 

#plt.xlim(-80, 70)
plt.xlabel('Potential INEXUS ($)', fontsize=32)
plt.ylabel('Density', fontsize=32)
ax.yaxis.set_tick_params(labelsize = 22)
ax.xaxis.set_tick_params(labelsize = 22)
#plt.savefig('baseline_mand.svg', format='svg')
plt.savefig('density_baseline.svg', bbox_extra_artists=(ax.legend(['Baseline Scenario'], loc="best", fontsize=34),), bbox_inches='tight')
plt.savefig('density_baseline.png', bbox_extra_artists=(ax.legend(['Baseline Scenario'], loc="best", fontsize=34),), bbox_inches='tight', dpi = 300)

#### Mode split Potential INEXUS

In [ ]:
sns.set(rc={'figure.figsize':(20,14)})
sns.set_context('talk')

sns.set_style("whitegrid", {'gridcolor': 'grainsboro'})
color_dict = {'walk/bike': '#CC3311', 'car': '#BBCC33', 'transit': '#009988', 'ride_hail': '#F4A582'} 

ax=sns.kdeplot(data=sfbase_clean_ml_walk, x="Potential_INEXUS_in_dollar_2023", hue = 'mode_choice_actual_4', 
            fill=True, common_norm=False, common_grid = False, palette=color_dict.values(), alpha=0.1, bw_adjust =15, linewidth =5)  #color_dict.values()
#sns.color_palette('rocket_r', n_colors=4) #palette=color_dict.values()
#rainbow_r  tab20
#sns.move_legend(ax, labels = ['Lowest 10% Income','Highest 10% Income'], loc="right", bbox_to_anchor=(1.25, 0.6), ncol=1, title=None, frameon=True)

# Define function to format y-axis labels as percentages
def to_percent(y, position):
    return "{:.1f}%".format(y * 100)

# Format y-axis labels as percentages
formatter = FuncFormatter(to_percent)
ax.yaxis.set_major_formatter(formatter)

#plt.xlim(-130, 130)

plt.legend(labels = ['Ridehail', 'Transit', 'Walk/Bike', 'Car'],
          fontsize='small', fancybox=False, ncol=1, frameon=True, loc = 'best', prop={'size': 32}) #title="Modes" #bbox_to_anchor=(1.5, 0.7), 

plt.xlabel('Potential INEXUS ($)', fontsize=32)
plt.ylabel('Density', fontsize=32)
ax.yaxis.set_tick_params(labelsize = 22)
ax.xaxis.set_tick_params(labelsize = 22)
#plt.savefig('baseline_mand.svg', format='svg')
plt.savefig('density_mode_baseline.svg', bbox_extra_artists=(ax.legend( ['Ridehail', 'Transit', 'Walk/Bike', 'Car'], loc="best", fontsize=34),), bbox_inches='tight')
plt.savefig('density_mode_baseline.png', bbox_extra_artists=(ax.legend( ['Ridehail', 'Transit', 'Walk/Bike', 'Car'], loc="best", fontsize=34),), bbox_inches='tight', dpi = 300)

#### Mandatory - Non-mandatory categories

In [ ]:
# Add the mandatory category column
mandatory = ['work' , 'univ', 'school']
sfbase_clean_ml_walk['mandatoryCat'] = np.where((sfbase_clean_ml_walk.actEndType.isin(mandatory)) & (sfbase_clean_ml_walk.actStartType.isin(mandatory)), 'from_M_to_M' , None)

In [ ]:
sfbase_clean_ml_walk['mandatoryCat'] = np.where((sfbase_clean_ml_walk.actEndType == 'Home') & (sfbase_clean_ml_walk.actStartType.isin(mandatory)), 'from_H_to_M' , sfbase_clean_ml_walk['mandatoryCat'])

In [ ]:
sfbase_clean_ml_walk['mandatoryCat'] = np.where((sfbassfbase_clean_ml_walke_clean_ml.actEndType.isin(mandatory)) & (sfbase_clean_ml_walk.actStartType == "Home"), 'from_M_to_H' , sfbase_clean_ml_walk['mandatoryCat'])

In [ ]:
non_mandatory = ['othmaint' , 'othdiscr', 'escort', 'eatout', 'social', 'shopping', 'atwork']
sfbase_clean_ml_walk['mandatoryCat'] = np.where((sfbase_clean_ml_walk.actEndType.isin(non_mandatory)) & (sfbase_clean_ml_walk.actStartType.isin(non_mandatory)), 'from_N_to_N' , sfbase_clean_ml_walk['mandatoryCat'])

In [ ]:
sfbase_clean_ml_walk['mand_cat'] = np.where(sfbase_clean_ml_walk['mandatoryCat'].isin(['from_M_to_M', 'from_M_to_H', 'from_H_to_M']), 'mand', 'non-mand')

In [ ]:
sfbase_clean_ml_walk.groupby('mand_cat')['Potential_INEXUS_in_dollar_2023'].describe()

In [ ]:
sns.set(rc={'figure.figsize':(20,14)}) #
sns.set_context('talk')

sns.set_style("whitegrid", {'gridcolor': 'grainsboro'})

ax=sns.kdeplot(data=sfbase_clean_ml_walk, x="Potential_INEXUS_in_dollar_2023", hue = 'mand_cat',
            fill=True, common_norm=False, alpha=0.1, palette=sns.color_palette('mako', n_colors=2), bw_adjust =15, linewidth =5)
sns.move_legend(ax, labels = ['Mandatory trips','Non-mandatory trips'], loc="best", ncol=1, title=None, frameon=True, fontsize = 28)#, bbox_to_anchor=(1.25, 0.6), 

# Define function to format y-axis labels as percentages
def to_percent(y, position):
    return "{:.1f}%".format(y * 100)

# Format y-axis labels as percentages
formatter = FuncFormatter(to_percent)
ax.yaxis.set_major_formatter(formatter)

#plt.xlim(-90, 75)
plt.xlabel('Potential INEXUS ($)', fontsize=32)
plt.ylabel('Density', fontsize=32)
ax.yaxis.set_tick_params(labelsize = 22)
ax.xaxis.set_tick_params(labelsize = 22)
#plt.savefig('baseline_mand.svg', format='svg')
plt.savefig('density_mand_baseline.svg', bbox_extra_artists=(ax.legend(['Mandatory trips','Non-mandatory trips'], loc="best", fontsize=28),), bbox_inches='tight')
plt.savefig('density_mand_baseline.png', bbox_extra_artists=(ax.legend(['Mandatory trips','Non-mandatory trips'], loc="best", fontsize=34),), bbox_inches='tight', dpi = 300)

#### Distance INEXUS plot

In [ ]:
# get the quartiles of the distance column
quartiles = np.percentile(sfbase_clean_ml['distance_travelling'], [25, 50, 75])

# set the cutoffs based on the quartiles
cutoffs = [0, quartiles[0], quartiles[1], quartiles[2], float('inf')]

# label the distances based on the cutoffs
sfbase_clean_ml['distance_group'] = pd.qcut(sfbase_clean_ml['distance_travelling'], q=3, labels=['short', 'medium', 'long'])

In [ ]:
# Add a column of income deciles
deciles = sfbase_clean_ml['distance_travelling'].quantile([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]).tolist()

In [ ]:
# Add distance_travelling deciles
conditions  = [(sfbase_clean_ml['distance_travelling'] >= deciles[0]) & (sfbase_clean_ml['distance_travelling'] < deciles[1]), 
               (sfbase_clean_ml['distance_travelling'] >= deciles[1]) & (sfbase_clean_ml['distance_travelling'] < deciles[2]),
               (sfbase_clean_ml['distance_travelling'] >=  deciles[2]) & (sfbase_clean_ml['distance_travelling'] < deciles[3]),
               (sfbase_clean_ml['distance_travelling'] >= deciles[3]) & (sfbase_clean_ml['distance_travelling'] < deciles[4]), 
               (sfbase_clean_ml['distance_travelling'] >=  deciles[4]) & (sfbase_clean_ml['distance_travelling'] < deciles[5]),
               (sfbase_clean_ml['distance_travelling'] >=  deciles[5]) & (sfbase_clean_ml['distance_travelling'] < deciles[6]),
               (sfbase_clean_ml['distance_travelling'] >=  deciles[6]) & (sfbase_clean_ml['distance_travelling'] < deciles[7]),
               (sfbase_clean_ml['distance_travelling'] >=  deciles[7]) & (sfbase_clean_ml['distance_travelling'] < deciles[8]),
               (sfbase_clean_ml['distance_travelling'] >=  deciles[8]) & (sfbase_clean_ml['distance_travelling'] < deciles[9]),
               (sfbase_clean_ml['distance_travelling'] >=  deciles[9]) & (sfbase_clean_ml['distance_travelling'] <= deciles[10])]

choices = [ '1stD', '2ndD', '3rdD', 
           '4thD', '5thD', '6thD', '7thD', '8thD', '9thD','10thD']

In [ ]:
sfbase_clean_ml['distance_deciles'] = np.select(conditions, choices, default=None)

In [ ]:
sns.set(rc={'figure.figsize':(20,14)})
sns.set_context('talk')

sns.set_style("whitegrid", {'gridcolor': 'grainsboro'})
colors = ["#1a9850", "#d73027", "#FFA07A", "#b3de69"] 

#color_dict = {'walk/bike': '#CC3311', 'car': '#BBCC33', 'transit': '#009988', 'ride_hail': '#F4A582'} 
sns.set_palette(sns.color_palette(colors))

ax=sns.kdeplot(data=sfbase_clean_ml[(sfbase_clean_ml['distance_deciles'] == '1stD')|(sfbase_clean_ml['distance_deciles'] == '10thD')]
                                    , x="Potential_INEXUS_in_dollar_2023", hue = 'distance_deciles', 
            fill=True, common_norm=False, common_grid = False,alpha=0.05, bw_adjust =15, linewidth =5)  #color_dict.values()
#sns.color_palette('rocket_r', n_colors=4) #palette=color_dict.values()
#rainbow_r  tab20
#sns.move_legend(ax, labels = ['Lowest 10% Income','Highest 10% Income'], loc="right", bbox_to_anchor=(1.25, 0.6), ncol=1, title=None, frameon=True)

# Define function to format y-axis labels as percentages
def to_percent(y, position):
    return "{:.1f}%".format(y * 100)

# Format y-axis labels as percentages
formatter = FuncFormatter(to_percent)
ax.yaxis.set_major_formatter(formatter)

#plt.xlim(-120, 100)

plt.legend(labels = ['10% longest distance','10% shortest distance'],
        fontsize='small', title_fontsize = 30, fancybox=False, ncol=1, frameon=True, loc = 'best', prop={'size': 32}) #title="Modes" #bbox_to_anchor=(1.5, 0.7), 

plt.xlabel('Potential INEXUS ($)', fontsize=32)
plt.ylabel('Density', fontsize=32)
ax.yaxis.set_tick_params(labelsize = 22)
ax.xaxis.set_tick_params(labelsize = 22)
#plt.savefig('baseline_mand.svg', format='svg')
plt.savefig('density_baseline_distance.svg', bbox_extra_artists=(ax.legend(['10% longest distance','10% shortest distance'], loc="best", fontsize=34),), bbox_inches='tight')
plt.savefig('density_baseline_distance.png', bbox_extra_artists=(ax.legend(['10% longest distance','10% shortest distance'], loc="best", fontsize=34),), bbox_inches='tight', dpi = 300)

### IQR (an alternative way to remove the outliers that we didn't use)

In [ ]:
# Calculate the quartiles, IQR, and bounds for column 1
q1_1, q3_1 = np.percentile(sfbase['Potential_INEXUS_in_dollar_2023'], [25, 75])
iqr_1 = q3_1 - q1_1
lower_bound_1 = q1_1 - 1.5 * iqr_1
upper_bound_1 = q3_1 + 1.5 * iqr_1

# Calculate the quartiles, IQR, and bounds for column 2
q1_2, q3_2 = np.percentile(sfbase['Realized_INEXUS_in_dollar_2023'], [25, 75])
iqr_2 = q3_2 - q1_2
lower_bound_2 = q1_2 - 1.5 * iqr_2
upper_bound_2 = q3_2 + 1.5 * iqr_2

# Filter out the data points that are outliers in either column
sfbase_clean_iqr = sfbase[(sfbase['Potential_INEXUS_in_dollar_2023'] >= lower_bound_1) & (sfbase['Potential_INEXUS_in_dollar_2023'] <= upper_bound_1) &
        (sfbase['Realized_INEXUS_in_dollar_2023'] >= lower_bound_2) & (sfbase['Realized_INEXUS_in_dollar_2023'] <= upper_bound_2)]